In [ ]:
import numpy as np 
import pandas as pd

import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

In [ ]:
warnings.filterwarnings("ignore")

# About the dataset
Attribute Information: (classes: edible=e, poisonous=p)

1. **cap-shape**: bell=b,conical=c,convex=x,flat=f, knobbed=k,sunken=s

2. **cap-surface**: fibrous=f,grooves=g,scaly=y,smooth=s

3. **cap-color**: brown=n,buff=b,cinnamon=c,gray=g,green=r,pink=p,purple=u,red=e,white=w,yellow=y

4. **bruises**: bruises=t,no=f

5. **odor**: almond=a,anise=l,creosote=c,fishy=y,foul=f,musty=m,none=n,pungent=p,spicy=s

6. **gill-attachment**: attached=a,descending=d,free=f,notched=n

7. **gill-spacing**: close=c,crowded=w,distant=d

8. **gill-size**: broad=b,narrow=n

9. **gill-color**: black=k,brown=n,buff=b,chocolate=h,gray=g, green=r,orange=o,pink=p,purple=u,red=e,white=w,yellow=y

10. **stalk-shape**: enlarging=e,tapering=t

11. **stalk-root**: bulbous=b,club=c,cup=u,equal=e,rhizomorphs=z,rooted=r,missing=?

12. **stalk-surface-above-ring**: fibrous=f,scaly=y,silky=k,smooth=s

13. **stalk-surface-below-ring**: fibrous=f,scaly=y,silky=k,smooth=s

14. **stalk-color-above-ring**: brown=n,buff=b,cinnamon=c,gray=g,orange=o,pink=p,red=e,white=w,yellow=y

15. **stalk-color-below-ring**: brown=n,buff=b,cinnamon=c,gray=g,orange=o,pink=p,red=e,white=w,yellow=y

16. **veil-type**: partial=p,universal=u

17. **veil-color**: brown=n,orange=o,white=w,yellow=y

18. **ring-number**: none=n,one=o,two=t

19. **ring-type**: cobwebby=c,evanescent=e,flaring=f,large=l,none=n,pendant=p,sheathing=s,zone=z

20. **spore-print-color**: black=k,brown=n,buff=b,chocolate=h,green=r,orange=o,purple=u,white=w,yellow=y

21. **population**: abundant=a,clustered=c,numerous=n,scattered=s,several=v,solitary=y

22. **habitat**: grasses=g,leaves=l,meadows=m,paths=p,urban=u,waste=w,woods=d

In [ ]:
# Load the dataset and print a few records 
dat = pd.read_csv('/kaggle/input/mushroom-classification/mushrooms.csv', header=0)
dat.head()

In [ ]:
dat.shape # 8124 tuples and 23 attributes 

In [ ]:
 dat.info()

In [ ]:
target = dat['class']
dat = dat.drop('class', axis=1)

In [ ]:
dat.head()

In [ ]:
dat.nunique()

In [ ]:
# veil-type has only 1 type of value and can be dropped
dat = dat.drop('veil-type', axis=1)

# Preprocessing

In [ ]:
x = dat
y = target.astype('category').cat.codes
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x,y,test_size=0.2, random_state=20, stratify=y)

In [ ]:
x_train.shape, x_val.shape, y_train.shape, y_val.shape

In [ ]:
# Ordinal encoding using LabelEncoder from sklearn
encoder = LabelEncoder()
for col in x_train.columns:
    x_train[col] = encoder.fit_transform(x_train[col])
    x_val[col] = encoder.transform(x_val[col])

x_train.head()

In [ ]:
corr = x_train.corr()
plt.figure(1, figsize=(15,10))
sns.heatmap(corr, annot = True)

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
std = StandardScaler()
x_train_scaled = std.fit_transform(x_train)
x_val_scaled = std.transform(x_val)

# Building the Artificial Neural Network (Multilayer Perceptron) 

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.05)
loss_fn = tf.keras.losses.BinaryCrossentropy()
model.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['accuracy'])

In [ ]:
model.fit(x_train_scaled, y_train, epochs=5, batch_size=64, validation_data=(x_val_scaled, y_val))

In [ ]:
preds = model.predict(x_val_scaled)

In [ ]:
preds = np.round(preds)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
confusion_matrix(y_val, preds)

In [ ]:
print(classification_report(y_val, preds))